## Import libraries and setup

In [1]:
import pandas as pd
import numpy as np
import itertools

import datetime
from datetime import timedelta

from tqdm.notebook import tqdm

import plotly.graph_objects as go


In [2]:

init_cash = 10000.00

data_loc = "S://Docs//Personal//MAEVE//Data//"


## Custom functions

### Calc functions

In [3]:
def calc_MA(df, timeperiod):
    df[f'MA{timeperiod}'] = df['Close'].rolling(window=timeperiod).mean()
    return df

In [4]:
def hodl_dca_perf(df, init_cash):
    
    hodl_buy = round(init_cash / df['Close'][0], 8)
    df['hodl_sats'] = hodl_buy
    df['hodl_usd'] = df['hodl_sats'] * df['Close']

    dcaamt = init_cash // 500
    dcabuy = len(df) // 500

    df['tmp_rownum'] = list(range(1, len(df)+1))
    df['tmp_dcabuyind'] = np.where(df['tmp_rownum'] % dcabuy == 0, 1, 0)

    df['tmp_dcabuys'] = 0
    df['tmp_dcabuys'] = np.where(df['tmp_dcabuyind'] == 1, round(dcaamt / df['Close'][0], 8),
                                df['tmp_dcabuys'])

    df['dca_sats'] = df['tmp_dcabuys'].cumsum()
    
    df['tmp_dcanumbuys'] = df['tmp_dcabuyind'].cumsum()
    
    df['dca_usd'] = (df['dca_sats'] * df['Close']) + \
                    (init_cash - (df['tmp_dcanumbuys']*dcaamt))
    
    remCols = [col for col in df.columns if 'tmp' in col]
    df.drop(columns=remCols, inplace=True)
    
    return df


### Logging functions

In [5]:
def log_trade(row, pos, cash, sats, init_cash=100):

    df = pd.DataFrame()

    df['Datetime'] = [row['Datetime']]
    df['price'] = [row['Close']]
    df['tradeType'] = [pos]
    df['cash'] = [cash]
    df['sats'] = [sats]
    df['profit/loss'] = np.where(df['sats'] > 0,
                                 (df['sats']*df['price']) - init_cash, df['cash'] - init_cash)

    return df


In [6]:
def strategy_summary(df):

    showCols = ['Datetime', 'Close', 'hodl_usd', 'dca_usd', 'maeve_usd']
    results_df = pd.concat([df[showCols].head(1), df[showCols].tail(1)])
    results_df.index = ['Start', 'End']

    results_df = pd.concat([results_df, pd.DataFrame({'Datetime': ['', ''], 'Close': ['Profit/Loss', '%'],
                                                      'hodl_usd': [results_df['hodl_usd'].End - results_df['hodl_usd'].Start, str(round(((results_df['hodl_usd'].End - results_df['hodl_usd'].Start)*100/init_cash), 2)) + '%'],
                                                      'dca_usd': [results_df['dca_usd'].End - results_df['dca_usd'].Start, str(round(((results_df['dca_usd'].End - results_df['dca_usd'].Start)/init_cash)*100, 2)) + '%'],
                                                      'maeve_usd': [results_df['maeve_usd'].End - results_df['maeve_usd'].Start, str(round(((results_df['maeve_usd'].End - results_df['maeve_usd'].Start)*100/init_cash), 2)) + '%']})], ignore_index=True)

    results_df.index = ['Start', 'End', '', '']

    return results_df


In [7]:
# MA1, MA2, stoploss, streaklim, cooldown, trailing
def log_backtest(combo, timeframe, summary_df):
    
    temp = pd.DataFrame()
    
    for strategy_type in ['HODL','DCA','MAEVE']:

        if strategy_type == 'MAEVE':
            strategy_id = '-'.join(['MAEVE', combo[0], combo[1], 'stop_'+str(combo[2]), 'streak_' + str(combo[3]), 'cooldown_'+str(combo[4]), 'trail_'+str(int(combo[5]))])

        else:
            strategy_id = strategy_type + '-' + timeframe
                    
        pnl = float(summary_df[strategy_type.lower()+"_usd"].values[-1][:-1])
        
        temp_ = pd.DataFrame({
                            'strategy_id': [strategy_id], 
                            'strategy_type': [strategy_type], 
                            'timeframe': [timeframe],
                            'MA1': [combo[0] if strategy_type == 'MAEVE' else ""],
                            'MA2': [combo[1] if strategy_type == 'MAEVE' else ""], 
                            'stoploss': [combo[2] if strategy_type == 'MAEVE' else ""], 
                            'streaklim': [combo[3] if strategy_type == 'MAEVE' else ""], 
                            'cooldown': [combo[4] if strategy_type == 'MAEVE' else ""],
                            'trailing': [combo[5] if strategy_type == 'MAEVE' else ""], 
                            'profit/loss': [pnl]
                            })
        
        if len(temp)==0:
            temp = temp_
        else:
            temp = pd.concat([temp, temp_])
            temp = temp.reset_index(drop=True)
    
        
    return temp


### Plotting functions

In [8]:
def plot_strategy_comparison(df):
    # Create the line plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.Datetime, y=df['hodl_usd'], name='HODL'))
    fig.add_trace(go.Scatter(x=df.Datetime, y=df['dca_usd'], name='DCA'))
    fig.add_trace(go.Scatter(x=df.Datetime, y=df['maeve_usd'], name='MAEVE'))

    # Set the title and axis labels
    fig.update_layout(title='Strategy returns over time',
                    xaxis_title='Time',
                    yaxis_title='USD')

    return fig


In [9]:
def plot_strategy(df, trades_df, MA1, MA2):

    # Create a trace for the candle chart
    candle = go.Candlestick(x=df['Datetime'],
                            open=df['Open'],
                            high=df['High'],
                            low=df['Low'],
                            close=df['Close'])

    # Create a trace for the buy points
    buy = go.Scatter(x=trades_df.loc[trades_df.tradeType == "buy"]['Datetime'],
                     y=trades_df.loc[trades_df.tradeType == "buy"]['price'],
                     mode='markers',
                     name='Buy',
                     marker=dict(size=10, color='green'))

    # Create a trace for the sell points
    sell = go.Scatter(x=trades_df.loc[trades_df.tradeType == "sell"]['Datetime'],
                      y=trades_df.loc[trades_df.tradeType == "sell"]['price'],
                      mode='markers',
                      name='Sell',
                      marker=dict(size=10, color='red'))
    
    # Create a trace for the MA1
    ma1 = go.Scatter(x=df['Datetime'], y=df[MA1],
                    mode='lines', name='MA1',
                    line=dict(width=2, color='orange'))

    # Create a trace for the MA2
    ma2 = go.Scatter(x=df['Datetime'], y=df[MA2],
                    mode='lines', name='MA2',
                    line=dict(width=2, color='blue'))

    # Create the plot
    fig = go.Figure(data=[candle, buy, sell, ma1, ma2])
    fig.update_layout(yaxis=dict(autorange=True, scaleanchor='y',
                                 scaleratio=1, fixedrange=False))

    return fig


### Backtest functions

In [10]:
# Apply streak only for successive stoploss triggers
# If stoploss triggers immediately after cool down, apply another cooldown

# After stoploss, turn off buy signal till MAs converge again

In [11]:
def run_maeve_backtest(df, combo, mode='single'):
    
    MA1 = combo[0]
    MA2 = combo[1]
    stoploss = combo[2]
    streaklim = combo[3]
    cooldown = combo[4]
    trailing = combo[5]

    
    # columns=['strategy_id','strategy_type','timeframe','MA1', 'MA2', 'stoploss', 'streaklim', 'cooldown', 'profit/loss']
    backtest_df = pd.DataFrame()
    figs = {}
    figs_strat = {}
    summary = {}
    trades = {}

    for timeframe, timeframename in zip(timeframes, timeframenames):
        
        df = btc_df[timeframe].reset_index(drop=True)
        
        # Calculate HODL / DCA Performance
        df = hodl_dca_perf(df, init_cash=init_cash)
        
        # Initialize the strategy variables
        current_position = None  # "buy" or "sell"
        cash = init_cash  # Starting cash
        sats = 0  # Starting BTC

        # Strategy logging
        trades_df = pd.DataFrame()
        strat_sats = []
        strat_usd = []

        # Position management
        stop_price = 0
        orig_stop_price = 0
        streak = 0
        idle = 0
        
        # Trade logging 
        trade_trigger = []

        # Iterate over the rows of the dataframe
        for index, row in df.iterrows():
            
            ############
            # Cooldown
            ############
            
            if streak >= streaklim:
                
                idle +=1
                strat_sats.append(row_sats)
                strat_usd.append(row_usd)
                
                if idle >= cooldown:
                    streak = 0
                    idle = 0
                    
                continue
            
            
            #######################
            # Position management
            #######################
            
            # Trailing stop loss
            if trailing:
                new_stop = round((1-stoploss) * row['Close'], 2)
                if stop_price == orig_stop_price:
                    if new_stop > (stop_price * (1+stoploss)): stop_price = new_stop
                else:
                    if new_stop > (stop_price * (1+0.01)): stop_price = new_stop
            
            # Check stop loss trigger
            if row['Close'] < stop_price and current_position == "buy":
                
                # Update position
                current_position = "sell"
                cash = round(sats * row['Close'], 2)
                sats = 0
                # row_usd = cash

                # Log trade
                trades_df = pd.concat([trades_df, log_trade(row, current_position, cash, sats)])
                trade_trigger.append('STOP')
                
                # Update streak
                streak += 1
                

            ###############
            # BUY signal
            ###############
            
            # Check if the MA1 is higher than the MA2
            if row[MA1] > row[MA2]:
                # If we're not currently holding any BTC, buy BTC
                if current_position != "buy":
                    
                    # Update position
                    current_position = "buy"
                    sats = round(cash / row['Close'], 8)
                    cash = 0
                    # row_sats = sats
                    
                    # Position management
                    orig_stop_price = round((1-stoploss) * row['Close'], 2)
                    stop_price = round((1-stoploss) * row['Close'], 2)
                    
                    # Log trade
                    trades_df = pd.concat([trades_df, log_trade(row, current_position, cash, sats)])
                    trade_trigger.append('BUY')
            
            
            ###############
            # SELL signal  
            ###############
                    
            # Check if the MA1 is lower than the MA2
            elif row[MA1] < row[MA2]:
                # If we're currently holding BTC, sell
                if current_position == "buy":
                    
                    # Update position
                    current_position = "sell"
                    cash = round(sats * row['Close'], 2)
                    sats = 0
                    # row_usd = cash
                    
                    # Log trade
                    trades_df = pd.concat([trades_df, log_trade(row, current_position, cash, sats)])
                    trade_trigger.append('SELL')
                    
            
            # Record row
            row_sats = sats
            row_usd = cash
            strat_sats.append(row_sats) 
            row_usd = (row_sats * row['Close']) + (row_usd)
            strat_usd.append(row_usd)

        df['maeve_sats'] = strat_sats
        df['maeve_usd'] = strat_usd
        
        trades_df['trade_trigger'] = trade_trigger

        if mode == 'single':
            fig = plot_strategy_comparison(df)
            figs[timeframename] = fig
            
            fig_strat = plot_strategy(df, trades_df, MA1, MA2)
            figs_strat[timeframename] = fig_strat
            
            trades[timeframename] = trades_df
            
        summary_df = strategy_summary(df)
        summary[timeframename] = summary_df
        
        if len(backtest_df) == 0:
            backtest_df = log_backtest(combo, timeframename, summary_df)
        else:
            backtest_df = pd.concat([backtest_df, log_backtest(combo, timeframename, summary_df)])
        
        backtest_df = backtest_df.reset_index(drop=True)
        backtest_df.drop_duplicates(inplace=True)
        
    return backtest_df, figs, figs_strat, trades, summary


## Prepare data

In [12]:
path = data_loc + "BTC_price_1h.csv"
btc_df = pd.read_csv(path)

print(f"Data shape: {btc_df.shape}")

print(f"Date range: {btc_df.Datetime.min()} - {btc_df.Datetime.max()}")

Data shape: (17017, 7)
Date range: 2021-02-01 00:00:00+00:00 - 2023-01-20 13:00:00+00:00


In [13]:
# Calculate MA
MALst = [12, 20, 24, 30, 40, 48, 50, 60, 100, 200]

for MA in MALst:
    btc_df = calc_MA(btc_df, MA)

btc_df.shape


(17017, 17)

## Identify test periods

In [14]:
alltime = (btc_df['Datetime'] >= "2010-01-01")

########################
# Bull market
########################
# Feb 01, 2021 - Apr 15, 2021
# Jul 15, 2021 - Nov 15, 2021

bull_market1 = (btc_df['Datetime'] >= "2021-02-01") & (btc_df['Datetime'] <= "2021-04-15")
bull_market2 = (btc_df['Datetime'] >= "2021-07-15") & (btc_df['Datetime'] <= "2021-11-15")

########################
# Bear market
########################
# Apr 15, 2021 - Jul 15, 2021
# Nov 15, 2021 - Feb 01, 2022
# Apr 01, 2022 - Jul 01, 2022

bear_market1 = (btc_df['Datetime'] >= "2021-04-15") & (btc_df['Datetime'] <= "2021-07-15")
bear_market2 = (btc_df['Datetime'] >= "2021-11-15") & (btc_df['Datetime'] <= "2022-02-01")
bear_market3 = (btc_df['Datetime'] >= "2022-04-01") & (btc_df['Datetime'] <= "2022-07-01")

########################
# Accumulation/ flat
########################
# Jul 1, 2022 - Nov 1, 2022
# Dec 1, 2022 - Jan 1, 2023 

accum_market1 = (btc_df['Datetime'] >= "2022-07-01") & (btc_df['Datetime'] <= "2022-11-01")
accum_market2 = (btc_df['Datetime'] >= "2022-12-01") & (btc_df['Datetime'] <= "2023-01-01")

################
# Bearish news
################

# Luna / 3AC / Celcius
# May 1, 2022 - Jul 1, 2022
blackswan1 = (btc_df['Datetime'] >= "2022-05-01") & (btc_df['Datetime'] <= "2022-07-01")

# FTX
# Nov 1, 2022 - Dec 1, 2022
blackswan2 = (btc_df['Datetime'] >= "2022-11-01") & (btc_df['Datetime'] <= "2022-12-01")

###################
# Bullish news
###################

# Tesla buy in
# Feb 1, 2021 - Mar 1, 2021
blackswan3 = (btc_df['Datetime'] >= "2021-02-01") & (btc_df['Datetime'] <= "2021-03-01")

# Futures ETF approval
# Sep 15, 2021 - Nov 1, 2021
blackswan4 = (btc_df['Datetime'] >= "2021-09-15") & (btc_df['Datetime'] <= "2021-11-01")

##############################
# Low volume time periods
##############################


# All test timeframes
timeframes = [alltime, bull_market1, bull_market2, bear_market1, bear_market2, bear_market3, accum_market1,
            accum_market2, blackswan1, blackswan2, blackswan3, blackswan4]

timeframenames = ['alltime','bull_market1', 'bull_market2', 'bear_market1', 'bear_market2', 'bear_market3', 'accum_market1',
              'accum_market2', 'luna', 'ftx', 'tesla_buy', 'etf_approval']


## Review specific strategy

In [15]:
# (MA1, MA2, stoploss, streaklim, cooldown)
# combo = ('MA12', 'MA24', 0.05, 5, 24, True)

# 60.96
combo = ('MA12', 'MA20', 0.01, 2, 48, True)

# 76.99
# combo = ('MA30', 'MA24', 0.01, 2, 24, True)

sbacktest_df, sfigs, sfigs_strat, strades, ssummary = run_maeve_backtest(btc_df, combo)


In [16]:
sbacktest_df

,strategy_id,strategy_type,timeframe,MA1,MA2,stoploss,streaklim,cooldown,trailing,profit/loss
0,HODL-alltime,HODL,alltime,,,,,,,-35.23
1,DCA-alltime,DCA,alltime,,,,,,,-35.23
2,MAEVE-MA12-MA20-stop_0.01-streak_2-cooldown_48...,MAEVE,alltime,MA12,MA20,0.01,2,48,True,60.96
3,HODL-bull_market1,HODL,bull_market1,,,,,,,93.44
4,DCA-bull_market1,DCA,bull_market1,,,,,,,108.01
5,MAEVE-MA12-MA20-stop_0.01-streak_2-cooldown_48...,MAEVE,bull_market1,MA12,MA20,0.01,2,48,True,82.85
6,HODL-bull_market2,HODL,bull_market2,,,,,,,98.76
7,DCA-bull_market2,DCA,bull_market2,,,,,,,112.39
8,MAEVE-MA12-MA20-stop_0.01-streak_2-cooldown_48...,MAEVE,bull_market2,MA12,MA20,0.01,2,48,True,89.45
9,HODL-bear_market1,HODL,bear_market1,,,,,,,-47.64


In [96]:
# sbacktest_df

In [74]:
sfigs['alltime'].show()


In [114]:
strades['bear_market1'].head()

,Datetime,price,tradeType,cash,sats,profit/loss,tradetype
0,2021-04-15 09:00:00+00:00,62767.335938,buy,0.00,0.159319,9899.999694,BUY
0,2021-04-15 16:00:00+00:00,62928.882812,sell,10025.74,0.000000,9925.740000,SELL
0,2021-04-15 22:00:00+00:00,63395.390625,buy,0.00,0.158146,9925.740125,BUY
0,2021-04-16 04:00:00+00:00,61900.785156,sell,9789.37,0.000000,9689.370000,STOP
0,2021-04-16 04:00:00+00:00,61900.785156,buy,0.00,0.158146,9689.370235,BUY


In [115]:
bear1_df = strades['bear_market1']

bear1_df[(bear1_df['Datetime'] >= "2021-05-08") &
        (bear1_df['Datetime'] <= "2021-05-29")].head(20)

,Datetime,price,tradeType,cash,sats,profit/loss,tradetype
0,2021-05-09 03:00:00+00:00,58640.484375,sell,8904.14,0.000000,8804.140000,SELL
0,2021-05-09 04:00:00+00:00,58158.867188,buy,0.00,0.153100,8804.140014,BUY
0,2021-05-09 07:00:00+00:00,58046.464844,sell,8886.93,0.000000,8786.930000,SELL
0,2021-05-10 02:00:00+00:00,58962.289062,buy,0.00,0.150722,8786.930052,BUY
0,2021-05-10 09:00:00+00:00,57996.804688,sell,8741.41,0.000000,8641.410000,STOP
0,2021-05-10 09:00:00+00:00,57996.804688,buy,0.00,0.150722,8641.410055,BUY
0,2021-05-10 14:00:00+00:00,57205.804688,sell,8622.19,0.000000,8522.190000,STOP
0,2021-05-10 14:00:00+00:00,57205.804688,buy,0.00,0.150722,8522.189884,BUY
0,2021-05-12 17:00:00+00:00,54879.421875,sell,8271.55,0.000000,8171.550000,STOP
0,2021-05-14 08:00:00+00:00,50342.375000,buy,0.00,0.164306,8171.550239,BUY


In [76]:
sfigs_strat['bear_market1'].show()